### Splitting Rasters into Equal Pieces

#### Using GDAL Warp

In [1]:
from osgeo import gdal

In [52]:
dem = gdal.Open('data/dem/north_mexico_clip.tif')

In [53]:
gt = dem.GetGeoTransform()

x_min = gt[0]
y_max = gt[3]
pixel_width = gt[1]
pixel_height = gt[5]

In [5]:
array = dem.GetRasterBand(1).ReadAsArray()
dx, dy = array.shape
print(dx, dy)

6089 7812


In [8]:
x_max = x_min + dx * pixel_width
y_min = y_max + dy * pixel_height

print(x_max, y_min)

996369.0 3397005.0


In [9]:
print(x_min, y_max)

978102.0 3420441.0


In [12]:
import numpy as np

In [23]:
xdiv = 3
ydiv = 3
x_range = np.linspace(x_min, x_max, xdiv + 1)
y_range = np.linspace(y_max, y_min, ydiv + 1)
print(x_range)
print(y_range)

[978102. 984191. 990280. 996369.]
[3420441. 3412629. 3404817. 3397005.]


In [46]:
for i in range(xdiv):
    for j in range(ydiv):
        x0 = x_range[i]
        x1 = x_range[i+1]
        y0 = y_range[j]
        y1 = y_range[j+1]
        
        gdal.Warp(f'output/northern_mexico_dem_{i}_{j}.tif', dem,
                  outputBounds=(x0, y0, x1, y1), dstNodata=-9999)


### Merge Rasters

In [30]:
import glob
import subprocess

In [60]:
dem_list = glob.glob('output/northern_mexico_dem_[0-9]_[0-9].tif')

In [48]:
dem_list

['output/northern_mexico_dem_0_0.tif',
 'output/northern_mexico_dem_0_1.tif',
 'output/northern_mexico_dem_0_2.tif',
 'output/northern_mexico_dem_1_0.tif',
 'output/northern_mexico_dem_1_1.tif',
 'output/northern_mexico_dem_1_2.tif',
 'output/northern_mexico_dem_2_0.tif',
 'output/northern_mexico_dem_2_1.tif',
 'output/northern_mexico_dem_2_2.tif']

In [61]:
cmd = 'gdal_merge.py -ps 3 -3 -o northern_mexico_merged.tif'
subprocess.call(cmd.split() + dem_list)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [73]:
vrt = gdal.BuildVRT('merged.tif', dem_list, xRes=pixel_width, yRes=pixel_height)
#gdal.Translate('output/northern_mexico_merged.tif', vrt, xRes=3, yRes=-3)

ERROR 5: Invalid user resolution


In [65]:
vrt = None